In [ ]:
import folium
import pandas

In [ ]:
map = folium.Map(
    location=[48.98662340201759, -47.83750077063219],
    zoom_start=4,
    tiles='Stamen Terrain',
)

feature_group_my_life = folium.FeatureGroup(name='my_life')

markers = [
    {
        "location": [59.958067512526746, 30.33278486692358],
        "name": 'My home 2021'
    },
    {
        "location": [59.96014928173088, 30.32536469920391],
        "name": 'My work 2021'
    },
    {
        "location": [59.903192337007646, 30.31503558578621],
        "name": 'Kitty house 2021'
    },
]

for marker in markers:
    feature_group_my_life.add_child(folium.Marker(
        location=marker["location"],
        popup=marker['name'],
        icon=folium.Icon(
            color='pink',
            icon='glyphicon glyphicon-home',
        )
    ))

feature_group_volcanoes = folium.FeatureGroup(name='volcanoes')

volcanoes = pandas.read_csv('./Webmap_datasources/Volcanoes.csv')

html = """
Volcano name:<br>
<a href="https://www.google.com/search?q=%%22%s%%22" target="_blank">%s</a><br>
Height: %s m
"""

def color_producer(elev: int):
    if elev < 1000:
        return 'green'
    elif elev < 3000:
        return 'orange'
    else:
        return 'red'

for lat, lon, elev, name, in zip(volcanoes.LAT, volcanoes.LON, volcanoes.ELEV, volcanoes.NAME, ):
    iframe = folium.IFrame(html=html % (
        name, name, elev,
    ), width=200, height=100)
    feature_group_volcanoes.add_child(folium.CircleMarker(
        location=[lat, lon],
        popup=folium.Popup(iframe),
        radius=12,
        color='grey',
        fill_color=color_producer(elev),
        fill_opacity=0.6,
    ))

feature_group_population = folium.FeatureGroup(name='population')

geo_json = folium.GeoJson(
    open('./Webmap_datasources/world.json', 'r', encoding='utf-8-sig').read(),
    style_function=lambda x: {
        'fillColor': 'green' if x['properties']['POP2005'] < 10000000 else 'orange' if x['properties']['POP2005'] < 20000000 else 'red'
    }
)
feature_group_population.add_child(geo_json)

map.add_child(feature_group_my_life)
map.add_child(feature_group_volcanoes)
map.add_child(feature_group_population)
map.add_child(folium.LayerControl())
map.save('Map.html')


In [ ]:
volcanoes
# help(zip)